In [7]:
import geopandas as gpd
import pydeck as pdk
from shapely.geometry import box
import os
from pathlib import Path
import pandas as pd

def get_data_dir():
    """Return path to the raw data directory."""
    try: 
        current_file = Path(__file__)
    except:
        current_file = Path(os.getcwd())
    project_root = current_file.parent.parent.parent
    data_dir = project_root / "aba_flooding" / "data" / "raw"
    return data_dir

def load_geojson(file_name):
    """Load a GeoJSON file from the data directory."""
    data_dir = get_data_dir()
    file_path = os.path.join(data_dir, file_name)
    print(file_path)
    return gpd.read_file(file_path)

def load_gpkg(file_name, layer=None):
    """Load a GeoPackage file from the data directory, with optional layer name."""
    data_dir = get_data_dir()
    file_path = os.path.join(data_dir, file_name)
    if layer:
        return gpd.read_file(file_path, layer=layer)
    return gpd.read_file(file_path)

def load_terrain_data(file_name):
    """Load terrain data from GeoJSON or GPKG file."""
    if file_name.endswith('.geojson'):
        return load_geojson(file_name)
    elif file_name.endswith('.gpkg'):
        return load_gpkg(file_name)
    else:
        raise ValueError(f"Unsupported file format for {file_name}")


ImportError: cannot import name 'Transformer' from 'pyproj' (unknown location)

In [ ]:

# Filter for Copenhagen (approximate bounding box for central Copenhagen)
# def filter_copenhagen(gdf):
#     copenhagen_bbox = box(
#     return gdf[gdf.intersects(copenhagen_bbox)]

# Convert GeoDataFrame to Pydeck-compatible format
def gdf_to_pydeck(gdf):
    features = [
        {"type": "Feature", "geometry": row.geometry.__geo_interface__, "properties": {}} 
        for _, row in gdf.iterrows()
    ]
    return {"type": "FeatureCollection", "features": features}

# Visualize using Pydeck
def visualize_terrain(file_path):
    gdf = load_geojson(file_path)
    print(f"Data loaded from {file_path}")
    print(f"Number of features: {len(gdf)}")
    
    print("columns:", gdf.columns)
    print("geometry:", gdf.geometry)

    #gdf = filter_copenhagen(gdf)
    #print(f"Number of features: {len(gdf)}")
    
    pydeck_data = gdf_to_pydeck(gdf)
    
    layer = pdk.Layer(
        "GeoJsonLayer",
        data=pydeck_data,
        get_fill_color=[255, 165, 0, 100],  # Orange color with transparency
        stroked=False,
    )
    
    #         latitude=56.2639,
        # longitude=9.5018,
    view_state = pdk.ViewState(
        # latitude=55.67,
        # longitude=12.57,
        latitude=56.2639,
        longitude=9.5018,
        zoom=11,
    )
    
    return pdk.Deck(layers=[layer], initial_view_state=view_state)

In [ ]:
# Convert GeoDataFrame to Pydeck-compatible format
def gdf_to_pydeck(gdf):
    """Convert a GeoDataFrame to PyDeck format, ensuring WGS84 projection."""
    # Make a copy to avoid modifying the original
    gdf = gdf.copy()
    
    # Check CRS and reproject to WGS84 if needed
    if gdf.crs is None:
        print("Warning: GeoDataFrame has no CRS defined. Assuming EPSG:25832 (UTM Zone 32N / ETRS89)")
        gdf.set_crs(epsg=25832, inplace=True)
    
    if gdf.crs != "EPSG:4326":
        print(f"Reprojecting from {gdf.crs} to WGS84 (EPSG:4326)")
        gdf = gdf.to_crs(epsg=4326)
    
    # Extract properties in addition to geometry
    features = []
    for idx, row in gdf.iterrows():
        properties = {col: row[col] for col in gdf.columns if col != 'geometry'}
        
        # Handle NaN values which aren't JSON serializable
        for key, value in properties.items():
            if pd.isna(value):
                properties[key] = None
                
        feature = {
            "type": "Feature", 
            "geometry": row.geometry.__geo_interface__, 
            "properties": properties
        }
        features.append(feature)
    
    return {"type": "FeatureCollection", "features": features}

# Visualize using Pydeck
def visualize_terrain(file_path):
    gdf = load_terrain_data(file_path)
    print(f"Data loaded from {file_path}")
    print(f"Number of features: {len(gdf)}")
    
    print("columns:", gdf.columns)
    print("geometry type sample:", gdf.geometry.iloc[0].geom_type)
    print(f"Coordinate reference system: {gdf.crs}")
    
    # Enhanced PyDeck visualization
    pydeck_data = gdf_to_pydeck(gdf)
    
    elevation_property = None
    # Check for potential elevation columns
    for col in ['MINKOTE', 'MAXKOTE', 'elevation', 'height', 'z']:
        if col in gdf.columns:
            elevation_property = f'properties.{col}'
            break
    
    layer = pdk.Layer(
        "GeoJsonLayer",
        data=pydeck_data,
        opacity=0.8,
        stroked=True,
        filled=True,
        extruded=True,  # Enable 3D
        wireframe=True,
        get_elevation=elevation_property or 10,  # Use identified elevation or default
        elevation_scale=50,
        get_fill_color=[255, 165, 0, 100],  # Orange color with transparency
        get_line_color=[255, 100, 0],
        get_line_width=1,
        pickable=True,  # Enable tooltips
    )
    
    # Set view state to the center of the data
    center_lon = gdf.to_crs(epsg=4326).geometry.centroid.x.mean()
    center_lat = gdf.to_crs(epsg=4326).geometry.centroid.y.mean()
    
    view_state = pdk.ViewState(
        latitude=center_lat,
        longitude=center_lon,
        zoom=10,
        pitch=45,  # Tilt the map for better 3D view
        bearing=0
    )
    
    deck = pdk.Deck(
        layers=[layer], 
        initial_view_state=view_state,
        map_style='mapbox://styles/mapbox/light-v9',
        tooltip={"text": "Properties: {properties}"}
    )
    
    return deck

In [ ]:
terrain_file = "Terrain.geojson"
deck = visualize_terrain(terrain_file)

NameError: name 'load_terrain_data' is not defined

In [ ]:
deck.show()

KeyboardInterrupt: 